In [145]:
# Import the dependencies
import pandas as pd
import os
import numpy as np
import datetime
import time
from calendar import timegm

In [135]:
# File to Load 
supply_data_to_load = "ETL/CAISO-supply-20220316.csv"

In [136]:
# Read the CSV and store into a Pandas DataFrame
supply_data_df = pd.read_csv(supply_data_to_load)

supply_data_df.head()

,Supply 03/16/2022,0:00,0:05,0:10,0:15,0:20,0:25,0:30,0:35,0:40,...,23:15,23:20,23:25,23:30,23:35,23:40,23:45,23:50,23:55,Unnamed: 289
0,Renewables,6387,6514,6562,6575,6546,6512,6498,6325,6220,...,5658,5585,5515,5465,5429,5387,5340,5323,5245,NaN
1,Natural gas,5353,5197,5060,5024,4928,4858,4790,4805,4814,...,6337,6305,6119,6062,5915,5842,5848,5726,5724,NaN
2,Large hydro,1087,1004,869,872,873,875,873,888,911,...,1305,1351,1323,1316,1292,1277,1277,1247,1228,NaN
3,Imports,7819,7857,8022,8017,8051,8062,8010,7935,7830,...,8089,8070,8043,8091,8128,8156,8168,8255,8238,NaN
4,Batteries,-88,-3,-8,-18,21,-9,29,-4,6,...,95,86,97,27,88,59,53,-5,103,NaN


In [137]:
# Reset index to energy sources and transpose
supply_df_transpose = supply_data_df.set_index('Supply 03/16/2022').T
supply_df_transpose

Supply 03/16/2022,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other
0:00,6387.0,5353.0,1087.0,7819.0,-88.0,2127.0,10.0,0.0
0:05,6514.0,5197.0,1004.0,7857.0,-3.0,2127.0,11.0,0.0
0:10,6562.0,5060.0,869.0,8022.0,-8.0,2127.0,11.0,0.0
0:15,6575.0,5024.0,872.0,8017.0,-18.0,2126.0,10.0,0.0
0:20,6546.0,4928.0,873.0,8051.0,21.0,2128.0,11.0,0.0
...,...,...,...,...,...,...,...,...
23:40,5387.0,5842.0,1277.0,8156.0,59.0,2114.0,9.0,0.0
23:45,5340.0,5848.0,1277.0,8168.0,53.0,2115.0,9.0,0.0
23:50,5323.0,5726.0,1247.0,8255.0,-5.0,2115.0,9.0,0.0
23:55,5245.0,5724.0,1228.0,8238.0,103.0,2115.0,9.0,0.0


In [138]:
# Create date column
supply_df_transpose["Date"] = "2022-03-16"
supply_df_transpose.head()

Supply 03/16/2022,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date
0:00,6387.0,5353.0,1087.0,7819.0,-88.0,2127.0,10.0,0.0,2022-03-16
0:05,6514.0,5197.0,1004.0,7857.0,-3.0,2127.0,11.0,0.0,2022-03-16
0:10,6562.0,5060.0,869.0,8022.0,-8.0,2127.0,11.0,0.0,2022-03-16
0:15,6575.0,5024.0,872.0,8017.0,-18.0,2126.0,10.0,0.0,2022-03-16
0:20,6546.0,4928.0,873.0,8051.0,21.0,2128.0,11.0,0.0,2022-03-16


In [139]:
# Create time column 
supply_df_transpose["Time"] = supply_df_transpose.index
supply_df_transpose

Supply 03/16/2022,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time
0:00,6387.0,5353.0,1087.0,7819.0,-88.0,2127.0,10.0,0.0,2022-03-16,0:00
0:05,6514.0,5197.0,1004.0,7857.0,-3.0,2127.0,11.0,0.0,2022-03-16,0:05
0:10,6562.0,5060.0,869.0,8022.0,-8.0,2127.0,11.0,0.0,2022-03-16,0:10
0:15,6575.0,5024.0,872.0,8017.0,-18.0,2126.0,10.0,0.0,2022-03-16,0:15
0:20,6546.0,4928.0,873.0,8051.0,21.0,2128.0,11.0,0.0,2022-03-16,0:20
...,...,...,...,...,...,...,...,...,...,...
23:40,5387.0,5842.0,1277.0,8156.0,59.0,2114.0,9.0,0.0,2022-03-16,23:40
23:45,5340.0,5848.0,1277.0,8168.0,53.0,2115.0,9.0,0.0,2022-03-16,23:45
23:50,5323.0,5726.0,1247.0,8255.0,-5.0,2115.0,9.0,0.0,2022-03-16,23:50
23:55,5245.0,5724.0,1228.0,8238.0,103.0,2115.0,9.0,0.0,2022-03-16,23:55


In [140]:
# Reset index to start from 0 and remove index name
supply_df_transpose.reset_index(drop=True, inplace=True)
supply_df_transpose.rename_axis(None, axis=1, inplace=True)
supply_df_transpose.head()

,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time
0,6387.0,5353.0,1087.0,7819.0,-88.0,2127.0,10.0,0.0,2022-03-16,0:00
1,6514.0,5197.0,1004.0,7857.0,-3.0,2127.0,11.0,0.0,2022-03-16,0:05
2,6562.0,5060.0,869.0,8022.0,-8.0,2127.0,11.0,0.0,2022-03-16,0:10
3,6575.0,5024.0,872.0,8017.0,-18.0,2126.0,10.0,0.0,2022-03-16,0:15
4,6546.0,4928.0,873.0,8051.0,21.0,2128.0,11.0,0.0,2022-03-16,0:20


In [141]:
# Cast 'Time' column to string and append seconds
supply_df_transpose['Time'] = supply_df_transpose['Time'].apply(str) + ':00'
supply_df_transpose.iloc[0:120, 9] = "0" + supply_df_transpose.iloc[0:120, 9]
supply_df_transpose.head(120)

,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time
0,6387.0,5353.0,1087.0,7819.0,-88.0,2127.0,10.0,0.0,2022-03-16,00:00:00
1,6514.0,5197.0,1004.0,7857.0,-3.0,2127.0,11.0,0.0,2022-03-16,00:05:00
2,6562.0,5060.0,869.0,8022.0,-8.0,2127.0,11.0,0.0,2022-03-16,00:10:00
3,6575.0,5024.0,872.0,8017.0,-18.0,2126.0,10.0,0.0,2022-03-16,00:15:00
4,6546.0,4928.0,873.0,8051.0,21.0,2128.0,11.0,0.0,2022-03-16,00:20:00
...,...,...,...,...,...,...,...,...,...,...
115,16250.0,3689.0,764.0,1778.0,158.0,2120.0,10.0,0.0,2022-03-16,09:35:00
116,16251.0,3790.0,749.0,1620.0,94.0,2120.0,10.0,0.0,2022-03-16,09:40:00
117,16124.0,3925.0,756.0,1615.0,62.0,2121.0,10.0,0.0,2022-03-16,09:45:00
118,15950.0,3916.0,792.0,1601.0,56.0,2121.0,10.0,0.0,2022-03-16,09:50:00


In [142]:
# Add 'Date' and 'Time' columns together
supply_df_transpose['DateTime'] = supply_df_transpose['Date'] + 'T' + supply_df_transpose['Time']
supply_df_transpose.head()

,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime
0,6387.0,5353.0,1087.0,7819.0,-88.0,2127.0,10.0,0.0,2022-03-16,00:00:00,2022-03-16T00:00:00
1,6514.0,5197.0,1004.0,7857.0,-3.0,2127.0,11.0,0.0,2022-03-16,00:05:00,2022-03-16T00:05:00
2,6562.0,5060.0,869.0,8022.0,-8.0,2127.0,11.0,0.0,2022-03-16,00:10:00,2022-03-16T00:10:00
3,6575.0,5024.0,872.0,8017.0,-18.0,2126.0,10.0,0.0,2022-03-16,00:15:00,2022-03-16T00:15:00
4,6546.0,4928.0,873.0,8051.0,21.0,2128.0,11.0,0.0,2022-03-16,00:20:00,2022-03-16T00:20:00


In [143]:
datetime.fromisoformat('2022-03-16T00:00:00')

datetime.datetime(2022, 3, 16, 0, 0)

In [150]:
ts= int(datetime.datetime(2022, 3, 16, 0, 0).timestamp())
print(ts)

1647414000
